In [1]:
import pandas as pd
import glob
import os
import re
import math
import matplotlib.pyplot as plt
import numpy as np
from statistics import mean


In [2]:
def metric(path, r_string, type_string, timing=False):
    all_files = glob.glob(os.path.join(path, "*"+r_string+type_string+"*.csv"))
    for file in all_files:
        if os.path.getsize(file) == 0:
            all_files.remove(file)
    # print("Running Metric")
    print(type(all_files))
    results = dict()
    results[type_string] = dict()
    results['reg'] = dict()
    results['time_'+type_string] = dict()
    results['time_reg'] = dict()
    results['CPD_'+type_string] = dict()
    results['CPD_REG'] = dict()
    # results['minw'] = dict()
    # results['minw_REG'] = dict()
    gnn_results = dict()
    reg_results = dict()
    for gnn_metric in all_files:
        lf = gnn_metric.split("/")[-1]
        lf = lf.split(".")[0]
        name = lf.split("__"+type_string+"__")[0]
        # print(name)
    
        df = pd.read_csv(gnn_metric,index_col=None,header=0)
        max_key = df.tail(1).index.values[0]
        # print(df['Iteration'])
        # print("Test" , df['Iteration'][max_key])  
        gnn_results[name] = int(df['Iteration'][max_key])
        # print(df[].head())
    results[type_string] = gnn_results
    reg_results = dict()
    for gnn_metric in all_files:
        lf = gnn_metric.split("/")[-1]
        lf = lf.split(".")[0]
        name = lf.split("__"+type_string+"__")[0]
        # print(name)
        df = pd.read_csv(gnn_metric,index_col=None,header=0)
       
        reg_results[name] = sum(df['Time'])
    results['time_'+type_string] = reg_results
    # print(results)
    reg_results = dict()

    reg_results = dict()
    for gnn_metric in all_files:
        lf = gnn_metric.split("/")[-1]
        lf = lf.split(".")[0]
        name = lf.split("__"+type_string+"__")[0]
        # print(name)
        df = pd.read_csv(gnn_metric,index_col=None,header=0)
       
        reg_results[name] = max(df['CPD (ns)'])
    results['CPD_'+type_string] = reg_results
    # print(results)
    reg_results = dict()
    all_files = glob.glob(os.path.join(path, r_string+"*reg*.csv"))
    for file in all_files:
        if os.path.getsize(file) == 0:
            all_files.remove(file)
    for reg_metric in all_files:
        lf = reg_metric.split("/")[-1]
        lf = lf.split(".")[0]
        name = lf.split("__reg__")[0]
        # print(name)
        df = pd.read_csv(reg_metric,index_col=None,header=0)

        max_key = df.tail(1).index.values[0]
        reg_results[name] = df['Iteration'][max_key]
        # print(df[].head())  
    results['reg'] = reg_results

    reg_results = dict()
    for reg_metric in all_files:
        lf = reg_metric.split("/")[-1]
        lf = lf.split(".")[0]
        name = lf.split("__reg__")[0]
        # print(name)
        df = pd.read_csv(reg_metric,index_col=None,header=0)
        reg_results[name] = sum(df['Time'])
    results['time_reg'] = reg_results

    reg_results = dict()
    for reg_metric in all_files:
        lf = reg_metric.split("/")[-1]
        lf = lf.split(".")[0]
        name = lf.split("__reg__")[0]
        # print(name)
        df = pd.read_csv(reg_metric,index_col=None,header=0)
        reg_results[name] = max(df['CPD (ns)'])
    results['CPD_REG'] = reg_results
        # print(df[].head())

    # print(results)
    my_df = pd.DataFrame.from_dict(results)
    # my_df = my_df.drop("EArch__des")
    # print(my_df)

    values1 = list(my_df['reg'].values)
    values2 = list(my_df[type_string].values)
    reduction = list()
    for i in range(len(values1)):
        if values1[i] == -1 or values2[i] == -1:
            reduction.append(0)
        else:
            reduction.append((values1[i]- values2[i]) /values1[i])
    my_df['Reduction'] = reduction
    # my_df['Reduction'] = [(i - j / i) for i, j in zip(values1, values2)]
    # my_df['Reduction'] = (my_df['reg'].values - my_df[type_string].values)/my_df['reg'].values
    # my_df['Reduction']=  pd.Series(["{0:.2f}%".format(val * 100) for val in my_df['Reduction']], index = my_df.index)

    # print("{:2%}".format(my_df))
    my_df.dropna(inplace=True)
    # my_df = my_df.drop("EArch__des")
    print("Average: ", "{:.2%}".format(mean(my_df['Reduction'].values)))

    pd.set_option('expand_frame_repr', False)
    my_df['Reduction']=  pd.Series(["{0:.2f}%".format(val * 100) for val in my_df['Reduction']], index = my_df.index)
    my_df['time_reg']=  pd.Series(["{0:.2f}".format(val) for val in my_df['time_reg']], index = my_df.index)
    my_df['time_'+type_string]=  pd.Series(["{0:.2f}".format(val) for val in my_df['time_'+type_string]], index = my_df.index)
    my_df['CPD_REG']=  pd.Series(["{0:.2f}".format(val) for val in my_df['CPD_REG']], index = my_df.index)
    my_df['CPD_'+type_string]=  pd.Series(["{0:.2f}".format(val) for val in my_df['CPD_'+type_string]], index = my_df.index)
    
    print(my_df)

In [89]:
path = "/mnt/e/benchmarks/route_metrics/MCNC/"
r_string = "EArch*"
type_string = "gnn"
metric(path, r_string, type_string)

<class 'list'>
Average:  5.34%
                 gnn  reg time_gnn time_reg CPD_gnn CPD_REG Reduction
EArch__alu4       12   -1     3.76     0.19    6.15    6.18     0.00%
EArch__apex2      27   -1     4.11     0.35    6.37    6.38     0.00%
EArch__apex4      -1   -1     5.47     0.33    6.12    5.97     0.00%
EArch__bigkey      9   14     5.33     0.18    2.68    2.49    35.71%
EArch__clma       24   -1     8.75     1.02   12.60   12.46     0.00%
EArch__des        -1   -1     5.87     0.38    5.46    5.99     0.00%
EArch__diffeq      9   12     4.39     0.15    6.55    6.55    25.00%
EArch__dsip       12   -1     3.69     0.46    3.05    3.12     0.00%
EArch__elliptic   11   -1     6.06     0.37    8.72    8.73     0.00%
EArch__ex1010     14   -1     5.48     0.50    8.06    8.06     0.00%
EArch__ex5p       28   -1     3.77     0.19    5.83    5.70     0.00%
EArch__frisc      21   -1     5.14     0.47   12.83   12.83     0.00%
EArch__misex3    131   -1     7.65     0.29    6.31    5.71

In [4]:
import tensorflow as tf
import datetime

In [94]:
path = "/mnt/e/benchmarks/route_metrics/TITAN/"
r_string = "strat*"
type_string = "gnn"
metric(path, r_string, type_string)

<class 'list'>
Average:  -36.33%
                            gnn  reg time_gnn time_reg CPD_gnn CPD_REG Reduction
stratixiv_arch__alu4       51.0   31     7.40     1.95   10.57   10.16   -64.52%
stratixiv_arch__apex2      -1.0   47     6.21     4.54   10.66   10.67     0.00%
stratixiv_arch__apex4      98.0   45    16.25     4.55   10.17   10.57  -117.78%
stratixiv_arch__bigkey      4.0    4    21.16     0.17    7.87    7.78     0.00%
stratixiv_arch__clma       -1.0   -1    19.93    21.79   10.30    9.98     0.00%
stratixiv_arch__des        12.0  102    28.73    25.85   12.43   12.43    88.24%
stratixiv_arch__diffeq     53.0   40    12.34     4.11    7.53    7.54   -32.50%
stratixiv_arch__dsip      100.0   19    24.19     0.82    7.52    7.52  -426.32%
stratixiv_arch__elliptic   47.0   40   221.43   140.31   10.28    9.76   -17.50%
stratixiv_arch__ex1010     87.0   62    37.13    17.02   13.09   13.13   -40.32%
stratixiv_arch__ex5p      145.0   63    36.11     8.56   10.85   10.49  -130